# 网易云音乐数据获取

调用[网易云音乐AP](https://binaryify.github.io/NeteaseCloudMusicApi/#/)来获取网易云音乐的一些信息

In [57]:
import os
from lxml import etree
import lxml.html
from selenium import webdriver
from user_agent import generate_user_agent

import requests
from tqdm import tqdm
import time
import json
from collections import Counter
import pandas as pd

# 获取歌手的top50歌曲

In [30]:
class Artist(object):
    
    def __init__(self,artist_id):
        self.artist_id = artist_id
    
    # 获取该歌手的top50歌曲的id、名字、歌手
    def get_top50_songs(self):
        artist_id = self.artist_id
        url = f'http://localhost:3000/artist/top/song?id={artist_id}'
        r = requests.get(url)
        songs = r.json()['songs']
        song_ids = [i['id'] for i in songs]
        song_names = [i['name'] for i in songs]
        song_artists = [i['ar'][0]['name'] for i in songs]
        return list(zip(song_ids,song_names,song_artists))
    
    # 获取某首歌的歌词
    def get_lyric(self,song_id):
        lyric_url = f'http://localhost:3000/lyric?id={song_id}'
        r = requests.get(lyric_url)
        lyric = r.json()['lrc']['lyric']
#         print(lyric)
        return lyric
    
    # 获取歌曲名字和歌手名字
    def get_music_detail(self,song_id):
        song_detail = f'http://localhost:3000/song/detail?ids={song_id}'
        r = requests.get(song_detail)
        music_name = r.json()['songs'][0]['name']
        artist = r.json()['songs'][0]['ar'][0]['name']
        return music_name,artist
    
    # 获取音乐url，并下载为mp3文件
    def get_music_url(self,song_id):
        music_url = f'http://localhost:3000/song/url?id={song_id}'
        music_name,artist = self.get_music_detail(song_id)
        r = requests.get(music_url)
        music = r.json()['data'][0]['url']
        content = requests.get(music).content
        with open(f'{artist}_{music_name}.mp3','wb') as f:
            f.write(content)
        return music
    
    
    # 获取该歌手top50歌曲的所有歌词
    def get_lyrics(self):
        lyrics = {}
        songs = self.get_top50_songs()
        song_ids = [i[0] for i in songs]
        for song_id in song_ids:
            lyrics[song_id] = self.get_lyric(song_id)
        return lyrics

## 李荣浩的歌曲

In [3]:
ronghao = Artist(4292)

In [4]:
# 获取李荣浩的top50歌曲歌词
lyrics_lrh = ronghao.get_lyrics()

In [5]:
lyrics_lrh.keys()

dict_keys([1407551413, 1293886117, 133998, 407450223, 518686034, 31654343, 1415566662, 1318234987, 1318235595, 1418319771, 29710981, 1410490187, 27678655, 27731176, 31134193, 29764562, 428375722, 29764563, 1308818966, 31877243, 506092654, 407450218, 31134170, 570334069, 27731174, 31134197, 27731172, 502242110, 27731177, 511920347, 29764561, 1305547844, 29764564, 862130427, 519250024, 486323960, 407459147, 407459148, 515143802, 1309866644, 29764565, 400074326, 1318234982, 27731179, 479223381, 407459145, 493631465, 29764568, 134000, 1318234981])

In [6]:
# 获取李荣浩某首歌的歌名
ronghao.get_music_detail(1293886117)

('年少有为', '李荣浩')

In [146]:
# 获取年少有为的音乐URL，并下载音乐
ronghao.get_music_url(1293886117)

'http://m8.music.126.net/20200206211754/af9f94e92e9d9f6beaed4cd31abcad24/ymusic/030b/060b/035b/4a0c8e3d1c1afd37d18e4165494e9ef2.mp3'

# 获取歌手榜上歌手的歌曲列表

In [64]:
def get_top_artist_musics():
    # 获取歌手榜上歌手的top50歌曲列表
    url = 'http://localhost:3000/toplist/artist'
    artists = requests.get(url).json()['list']['artists']

    # 对歌手榜中的歌手，获取每个歌手的top50歌曲
    songs_list = []
    for ar in [i.get('id') for i in artists]:
        artist = Artist(ar)
        songs = artist.get_top50_songs()
    #     song_names = [i[1] for i in songs]
        songs_list.extend(songs)

    df_songs = pd.DataFrame(songs_list)
    df_songs.columns = ['id','name','artist']
    return df_songs



In [66]:
df_songs = get_top_artist_musics()

In [67]:
df_songs.head()

,id,name,artist
0,1346104327,多想在平庸的生活拥抱你,隔壁老樊
1,1383927243,这一生关于你的风景,隔壁老樊
2,1336856777,我曾,隔壁老樊
3,1365221826,四块五,隔壁老樊
4,1405319676,我的姑娘 她在远方,隔壁老樊


# 歌手榜中歌手的歌曲名字中的第一个字分析

In [68]:
first_words = [i[0] for i in df_songs['name']]

Counter(first_words).most_common(10)

[('我', 162),
 ('不', 88),
 ('你', 86),
 ('一', 73),
 ('爱', 49),
 ('小', 43),
 ('无', 42),
 ('S', 36),
 ('C', 35),
 ('I', 34)]

原来热门歌手在为他们的歌曲起名字时，喜欢以“我”、“不”、“你”作为开头

例如，以“我”开头的歌取有：

In [70]:
df_songs[df_songs['name'].apply(lambda x:x[0]=="我")].head(20)

,id,name,artist
2,1336856777,我曾,隔壁老樊
4,1405319676,我的姑娘 她在远方,隔壁老樊
26,549059804,我曾demo,隔壁老樊
31,1394169337,我和我的祖国,隔壁老樊
34,1327199767,我曾Demo,隔壁老樊
39,1336856778,我曾（伴奏）,隔壁老樊
47,1405318715,我的姑娘 她在远方（伴奏）,隔壁老樊
70,506092654,我看着你的时候,李荣浩
92,1318234982,我知道是你,李荣浩
109,233974,我的秘密,G.E.M.邓紫棋


In [72]:
mygirl = Artist(12429072).get_music_url(1405319676)

让我们一起听一下隔壁老樊的[我的姑娘 她在远方](http://m8.music.126.net/20200206225113/da1f24d78a7487687b701cf0fbce99d5/ymusic/5259/005e/0e5c/e3d8f29ed874fb50d44c96df59f0c6b2.mp3)